In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from juggling_dlx_milp import *

In [4]:
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [5]:
balls, throws = music_to_throws(music)

In [6]:
balls

{'do', 'mi', 're'}

In [7]:
throws

[[Throw(ball='do', time=0, max_height=1),
  Throw(ball='re', time=0, max_height=4),
  Throw(ball='mi', time=0, max_height=5)],
 [Throw(ball='do', time=1, max_height=1)],
 [Throw(ball='do', time=2, max_height=1)],
 [Throw(ball='do', time=3, max_height=6)],
 [Throw(ball='re', time=4, max_height=3)],
 [Throw(ball='mi', time=5, max_height=5)],
 [],
 [Throw(ball='re', time=7, max_height=4)],
 [],
 [Throw(ball='do', time=9, max_height=4)],
 [],
 [Throw(ball='re', time=11, max_height=1)],
 []]

In [8]:
ec_instance1 = throws_to_extended_exact_cover(balls, throws, 2, 4, 2,
                                             [(1, 3)], [(1, 3), (1, 1)], True)

In [9]:
sol1 = solve_exact_cover_with_milp(ec_instance1, True)

In [10]:
ec_instance2 = throws_to_extended_exact_cover(balls, throws, 2, 4, 2,
                                             [(1, 3)], [(1, 3), (1, 1)], False)

In [11]:
sol2 = solve_exact_cover_with_milp(ec_instance2)

#### Solution en interdisant de lancer plusieurs balles depuis une même main en même temps

In [12]:
print_juggling(sol1)

   {'re'}    {'do', 'mi'} : do -- 1 --> 0
{'do', 're'}    {'mi'}    : do -- 1 --> 1
   {'re'}    {'do', 'mi'} : re -- 2 --> 0
                            do -- 1 --> 0
   {'do'}       {'mi'}    : mi -- 2 --> 1
{'do', 're'}      {}      : 
{'do', 're'}    {'mi'}    : 
{'do', 're'}    {'mi'}    : re -- 1 --> 0
{'do', 're'}    {'mi'}    : 
{'do', 're'}    {'mi'}    : do -- 1 --> 0
{'do', 're'}    {'mi'}    : mi -- 1 --> ?
{'do', 're'}      {}      : re -- 1 --> 0
{'do', 're'}      {}      : re -- 1 --> ?
   {'do'}         {}      : do -- 1 --> ?


#### Solution en autorisant de lancer plusieurs balles depuis une même main en même temps

In [13]:
print_juggling(sol2)

{'do', 're'}    {'mi'}    : do -- 1 --> 1
   {'re'}    {'do', 'mi'} : do -- 1 --> 0
{'do', 're'}    {'mi'}    : re -- 2 --> 0
                            do -- 1 --> 0
   {'do'}       {'mi'}    : mi -- 2 --> 1
{'do', 're'}      {}      : 
{'do', 're'}    {'mi'}    : 
{'do', 're'}    {'mi'}    : re -- 1 --> 0
{'do', 're'}    {'mi'}    : 
{'do', 're'}    {'mi'}    : do -- 1 --> 0
{'do', 're'}    {'mi'}    : mi -- 1 --> ?
{'do', 're'}      {}      : re -- 1 --> 0
{'do', 're'}      {}      : re -- 1 --> ?
   {'do'}         {}      : do -- 1 --> ?


#### Même comparaison avec d'autres fonctions d'affichages

In [14]:
print_juggling_solution(sol1)

0 - do : main 1 (0 temps) --> 1 temps
0 - re : main 0 (2 temps) --> 2 temps
0 - mi : main 1 (3 temps) --> 2 temps
1 - do : main 0 (0 temps) --> 1 temps
2 - do : main 1 (0 temps) --> 1 temps
3 - do : main 0 (5 temps) --> 1 temps
4 - re : main 0 (2 temps) --> 1 temps
5 - mi : main 1 (4 temps) --> 1 temps
7 - re : main 0 (3 temps) --> 1 temps
9 - do : main 0 (3 temps) --> 1 temps
11 - re : main 0 (0 temps) --> 1 temps


In [15]:
print_juggling_solution(sol2)

0 - do : main 0 (0 temps) --> 1 temps
0 - re : main 0 (2 temps) --> 2 temps
0 - mi : main 1 (3 temps) --> 2 temps
1 - do : main 1 (0 temps) --> 1 temps
2 - do : main 0 (0 temps) --> 1 temps
3 - do : main 0 (5 temps) --> 1 temps
4 - re : main 0 (2 temps) --> 1 temps
5 - mi : main 1 (4 temps) --> 1 temps
7 - re : main 0 (3 temps) --> 1 temps
9 - do : main 0 (3 temps) --> 1 temps
11 - re : main 0 (0 temps) --> 1 temps


#### Export vers le format du simulateur

In [17]:
juggling_sol_to_simulator(sol1)

[[[],
  [('do', 1, 1)],
  [('re', 0, 2)],
  [],
  [],
  [],
  [('re', 0, 1)],
  [],
  [('do', 0, 1)],
  [],
  [('re', 0, 1)],
  [('re', 0, 1)],
  [('do', 0, 1)],
  []],
 [[('do', 0, 1)],
  [],
  [('do', 0, 1)],
  [('mi', 1, 2)],
  [],
  [],
  [],
  [],
  [],
  [('mi', 0, 1)],
  [],
  [],
  [],
  []]]

Une éventuelle contrainte à rajouter : interdire qu'une main ne lance et rattrape une balle au même moment.

...

Après réflexion ce n'est en fait pas nécessaire. Si on veut jongler à la suite 3 lancers de hauteur 1 avec deux mains on va faire : main 0 -> main 1 -> main 0 -> main 1.

Formellement une main relance une balle au même instant où elle la reçoit. En réalité, elle reçoit la balle puis la garde un court instant dans la main (moins d'un temps) puis la relance. Une main ne lance et ne reçoit donc jamais en même temps. Le temps où la balle reste dans la main entre la réception et le lancer par le jongleur fait partie du temps de vol dans le modèle.

## Notes suite à la discussion du 7/05

### Contraintes supplémentaires à ajouter à la réduction
- interdir de lancer deux balles en même temps dans la même main
- score de jouabilité (minimiser le nombre de lancer en même temps)
- temps de départ à prendre en compte

### Corrections à faire
- corriger la réduction pour le cas où on a des lancers multiplex où les balles étaient déjà dans la main (et pas : elles arrivent puis sont relancées immédiatement après)

### Changements à essayer dans l'algorithme
- utiliser l'algo avec les polytopes pour générer toutes les solutions (cf. la solution de cette [question](https://ask.sagemath.org/question/23666/solving-system-of-linear-diophantine-equations/))

### Autres choses pratiques à faire
- améliorer un peu la classe `Audio` (définir à l'initialisation du programme une fonction javascript qui est ensuite simplement appelée par python)

In [20]:
# des erreurs vont être affichées mais c'est bon en fait
generate_full_table(ec_instance)

Rc files read:
  NONE
Latexmk: This is Latexmk, John Collins, 29 September 2020, version: 4.70b.
Rule 'pdflatex': File changes, etc:
   Changed files, or newly in use since previous run(s):
      '/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex'
      'full_table.tex'
------------
Run number 1 of rule 'pdflatex'
------------
------------
Running 'pdflatex  --interaction=nonstopmode -recorder  "/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex"'
------------
Latexmk: applying rule 'pdflatex'...
This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020/Arch Linux) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex
LaTeX2e <2020-10-01> patch level 2
L3 programming layer <2020-12-03> xparse <2020-03-03>
(/usr/share/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/us

CalledProcessError: Command '['latexmk', '--pdf', '--interaction=nonstopmode', '/home/josue/Documents/git/Jugglyng1/Expérimentations/full_table.tex']' returned non-zero exit status 12.

In [21]:
generate_table(ec_instance)